In [ ]:
import pandas as pd

# Load the data from the Excel files
df_2018_2019 = pd.read_excel('PAE2018-2019.xlsx')
df_2019_2020 = pd.read_excel('PAE2019-2020.xlsx')
df_2021_2022 = pd.read_excel('PAE2021-2022.xlsx')
df_2022_2023 = pd.read_excel('PAE2022-2023.xlsx')

# Explore the data
df_2018_2019.info()
df_2019_2020.describe()
df_2021_2022.head()

# Select specific columns
columns_to_select = ['Qual a sua REMUNERAÇÃO BRUTA mensal em termos de salários mínimos (referência: 2018)?',
                     'Atualmente você trabalha na área de formação e conhecimentos obtidos no IFRN?',
                     'AUX - Situação trabalho/estudos',
                     'Campus do IFRN de realização dos estudos:',
                     'CURSO',
                     'ANO_CONCLUSAO']
df_2018_2019_selected = df_2018_2019[columns_to_select]
df_2019_2020_selected = df_2019_2020[columns_to_select]
df_2021_2022_selected = df_2021_2022[columns_to_select]

# Save the selected DataFrames
df_2018_2019_selected.to_excel('PAE2018-2019_selected_data.xlsx', index=False)
df_2019_2020_selected.to_excel('PAE2019-2020_selected_data.xlsx', index=False)
df_2021_2022_selected.to_excel('PAE2021-2022_selected_data.xlsx', index=False)

# Load the saved DataFrames
df_2018_2019_selected = pd.read_excel('PAE2018-2019_selected_data.xlsx')
df_2019_2020_selected = pd.read_excel('PAE2019-2020_selected_data.xlsx')
df_2021_2022_selected = pd.read_excel('PAE2021-2022_selected_data.xlsx')

# Create a dictionary mapping course names to their modalities
course_to_modalities = df_2019_2020_selected.set_index('CURSO')['MODALIDADE'].to_dict()

# Add a new column 'MODALIDADE' to 'df_2018_2019_selected' by mapping the 'CURSO' column using the 'course_to_modalities' dictionary
df_2018_2019_selected['MODALIDADE'] = df_2018_2019_selected['CURSO'].map(course_to_modalities)

# Count the number of missing values in the new 'MODALIDADE' column
missing_values_count = df_2018_2019_selected['MODALIDADE'].isna().sum()

# Update the 'MODALIDADE' column based on the rules
df_2018_2019_selected.loc[df_2018_2019_selected['CURSO'].str.startswith('Especialização'), 'MODALIDADE'] = 'Especialização'
df_2018_2019_selected.loc[df_2018_2019_selected['CURSO'].str.startswith('Aperfeiçoamento'), 'MODALIDADE'] = 'Aperfeiçoamento'
df_2018_2019_selected.loc[df_2018_2019_selected['CURSO'].str.startswith('Técnico'), 'MODALIDADE'] = 'Técnico Subsequente'
df_2018_2019_selected.loc[df_2018_2019_selected['CURSO'].str.startswith('Tecnologia'), 'MODALIDADE'] = 'Tecnologia'
df_2018_2019_selected.loc[df_2018_2019_selected['CURSO'].str.startswith('Mestrado'), 'MODALIDADE'] = 'Mestrado'

# Count the number of missing values in the 'MODALIDADE' column after updating
missing_values_count_updated = df_2018_2019_selected['MODALIDADE'].isna().sum()

# Concatenate the DataFrames
df_unified = pd.concat([df_2018_2019_selected, df_2019_2020_selected, df_2021_2022_selected], ignore_index=True)

# Add the 'NOME_DATAFRAME' column to identify the source of each row
df_unified['NOME_DATAFRAME'] = pd.Series(['PAE2018-2019'] * len(df_2018_2019_selected) +
                                         ['PAE2019-2020'] * len(df_2019_2020_selected) +
                                         ['PAE2021-2022'] * len(df_2021_2022_selected))

# Save the unified DataFrame to a CSV file
df_unified.to_csv('PAE-unificado.csv', index=False)


In [2]:
%%writefile "drive/MyDrive/Colab Notebooks/EGRESSOS/app_st.py"

# Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Get a summary of the DataFrame
df_summary = df_unified.describe(include='all')

# Count the number of missing values in each column
missing_values = df_unified.isna().sum()

# Count the number of unique values in each column
unique_values = df_unified.nunique()

# Plot the distribution of each column
for col in df_unified.columns:
    if df_unified[col].dtype in ['int64', 'float64']:
        plt.figure(figsize=(10, 5))
        sns.histplot(data=df_unified, x=col, kde=True)
        plt.title(f'Distribution of {col}')
        plt.show()
    else:
        plt.figure(figsize=(10, 5))
        sns.countplot(data=df_unified, y=col, order=df_unified[col].value_counts().index)
        plt.title(f'Count of each category in {col}')
        plt.show()

df_summary, missing_values, unique_values


Writing drive/MyDrive/Colab Notebooks/EGRESSOS/app_st.py


In [3]:
!pip install pipreqs

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=057ce0bbc07b26c67261172ae254b08a567190ec894c6100ad8ee2abce6e1d16
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [4]:
!pipreqs drive/MyDrive/Colab\ Notebooks/EGRESSOS/ --savepath drive/MyDrive/Colab\ Notebooks/EGRESSOS/requirements.txt --force

Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in drive/MyDrive/Colab Notebooks/EGRESSOS/requirements.txt
